# Let's try to get the newest CNN working for regressions too

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
#import libraries and some constants

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize, LogNorm
import pandas as pd
import uproot as ur
import atlas_mpl_style as ampl
ampl.use_atlas_style()

path_prefix = '/g/g16/heinz6/workspace/projects/caloml-atlas/code/caloml-atlas-py/'

In [5]:
# import our resolution utilities

import sys
sys.path.append(path_prefix)
from  util import resolution_util as ru
from  util import plot_util as pu
from  util import ml_util as mu

Using TensorFlow backend.


In [6]:
inputpath = '/usr/WS1/hip/ML4Jets/regression_images/images_v6_v01-17-g2679fe7/'
file_list = [inputpath+'piplus_unscaled_filtered.root', inputpath+'piplus.root']

In [26]:
data = ur.iterate(file_list, 'ClusterTree', 'cluster_ENG_CALIB_TOT', entrysteps = [(0, 10)], namedecode = 'utf-8')

data1 = list(data)
print(len(data1))
print(data1)

2
[{'cluster_ENG_CALIB_TOT': array([1.79040479e+03, 7.33295059e+01, 2.59032995e-01, 1.29555616e+01,
       1.16086915e-01, 1.11815527e+03, 3.03148198e+00, 1.85991037e+00,
       2.27923346e+00, 5.76284027e+00], dtype=float32)}, {'cluster_ENG_CALIB_TOT': array([1.79040479e+03, 7.33295059e+01, 2.59032995e-01, 1.29555616e+01,
       1.16086915e-01, 1.11815527e+03, 3.03148198e+00, 1.85991037e+00,
       2.27923346e+00, 5.76284027e+00], dtype=float32)}]


In [14]:
len(data1[7]['cluster_ENG_CALIB_TOT'])

57525

In [26]:
print(len(data1[3]['cluster_ENG_CALIB_TOT']))

47377


In [27]:
#inputpath = '/usr/WS1/hip/ML4Jets/regression_images/sample_jet_events/'
#rootfiles = ["mltree.pool"]
inputpath = '/usr/WS1/hip/ML4Jets/regression_images/images_v6_v01-17-g2679fe7/'
rootfiles = ['piplus_unscaled_filtered']

branches = ['TileBar2', 'EMB3', 'cluster_ENG_CALIB_TOT', 'cluster_sumCellE',\
            'TileBar1', 'TileBar0', 'cluster_nCells', 'clusterPt', 'clusterIndex',\
            'clusterEta', 'EMB1', 'clusterPhi', 'EMB2', 'clusterE', 'truthE']

trees, pdata = mu.setupPionData(inputpath, rootfiles, branches)

In [28]:
tree = trees['piplus_unscaled_filtered']

In [30]:
import random

num_points = 10
batch_size = 256
start_points = np.array([random.randrange(0,batch_size*num_points) for idx in range(num_points)])
print(start_points)

[2412 1560 1386 1498 2015 1053 2465  444 1154  990]


In [31]:
data = tree.iterate(branches, entrysteps = [(0,2),(2,4)],
                   namedecode = 'utf-8')
data1 = list(data)
print(data1[1]['cluster_ENG_CALIB_TOT'])

data = tree.iterate(branches, entrysteps = [(0,4)],
                   namedecode = 'utf-8')
data1 = list(data)
print(data1[0]['cluster_ENG_CALIB_TOT'])

data = tree.iterate(branches, entrystart=0, entrystop = 4,
                   namedecode = 'utf-8')
data2 = list(data)
print(data2[0]['cluster_ENG_CALIB_TOT'])

[ 0.259033 12.955562]
[1.7904048e+03 7.3329506e+01 2.5903299e-01 1.2955562e+01]
[1.7904048e+03 7.3329506e+01 2.5903299e-01 1.2955562e+01]


In [39]:
for idx in range(num_points):
    data = tree.iterate(branches, entrysteps=[(start_points[idx], start_points[idx]+ batch_size)], namedecode='utf-8')
    data1 = list(data)
    
    print(len(data1), data1[0]['cluster_ENG_CALIB_TOT'].shape, data1[0]['cluster_ENG_CALIB_TOT'][[0,1]])

1 (256,) [404.05792 369.35486]
1 (256,) [142.551        0.36688793]
1 (256,) [  2.2390928 176.55815  ]
1 (256,) [1.5416069  0.38817537]
1 (256,) [121.27202 252.69635]
1 (256,) [0.9807285 1.2521334]
1 (256,) [381.81512   14.379922]
1 (256,) [  0.16409473 102.14722   ]
1 (256,) [137.79831     4.8507814]
1 (256,) [2.1841986  0.22467041]


In [45]:
for idx in range(num_points):
    data = tree.iterate(branches, entrystart=start_points[idx], 
                        entrystop=start_points[idx]+batch_size, entrysteps=batch_size, namedecode='utf-8')
    data1 = list(data)
    
    print(len(data1), data1[0]['EMB1'].shape, data1[0]['cluster_ENG_CALIB_TOT'][[0,1]])

1 (256, 128, 4) [404.05792 369.35486]
1 (256, 128, 4) [142.551        0.36688793]
1 (256, 128, 4) [  2.2390928 176.55815  ]
1 (256, 128, 4) [1.5416069  0.38817537]
1 (256, 128, 4) [121.27202 252.69635]
1 (256, 128, 4) [0.9807285 1.2521334]
1 (256, 128, 4) [381.81512   14.379922]
1 (256, 128, 4) [  0.16409473 102.14722   ]
1 (256, 128, 4) [137.79831     4.8507814]
1 (256, 128, 4) [2.1841986  0.22467041]


In [8]:
%%timeit

img_data = tree.lazyarrays(branches = branches, namedecode = 'utf-8')

for idx in range(num_points):
    temp_idxs = np.arange(idx*batch_size, (idx+1)*batch_size)
    emb1 = img_data['EMB1'][temp_idxs]
    print(type(emb1),emb1.shape)

<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 4)
<class 'awkward.array.chunked.ChunkedArray'> (256, 128, 

In [9]:
%%timeit

img_data = tree.lazyarrays(branches = branches, namedecode = 'utf-8')

for idx in range(num_points):
    temp_idxs = np.arange(idx*batch_size, (idx+1)*batch_size)
    emb1 = np.array(list(img_data['EMB1'][temp_idxs]))
    print(type(emb1),emb1.shape)

<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'nump

In [23]:
img_data = tree.lazyarrays(branches = branches, namedecode = 'utf-8')

In [25]:
%%timeit -r1 -n1

for idx in range(num_points):
    temp_idxs = np.arange(idx*batch_size, (idx+1)*batch_size)
    emb1 = np.array(list(img_data['EMB1'][temp_idxs]))
    print(type(emb1),emb1.shape)

<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
<class 'numpy.ndarray'> (256, 128, 4)
14.8 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [12]:
%%timeit

img_data = tree.lazyarrays(branches = branches, namedecode = 'utf-8')

30.3 ms ± 269 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
